In [9]:
import pandas as pd

# File paths
hitters_file = "/Users/stephenak24/Downloads/2023_hitters.csv"
merged_success_file = "/Users/stephenak24/Downloads/merged_success_v2.csv"
oppo_fb_file = "/Users/stephenak24/Downloads/oppo_flyball_2023.csv"
pulled_fb_file = "/Users/stephenak24/Downloads/pulled_flyball_2023.csv"
straightaway_fb_file = "/Users/stephenak24/Downloads/straightaway_flyball_2023.csv"
owar_file = "/Users/stephenak24/Downloads/baseball_2023_war.csv"

hitters_data = pd.read_csv(hitters_file)
merged_success_data = pd.read_csv(merged_success_file)
oppo_fb_data = pd.read_csv(oppo_fb_file)
pulled_fb_data = pd.read_csv(pulled_fb_file)
straightaway_fb_data = pd.read_csv(straightaway_fb_file)
owar_data = pd.read_csv(owar_file)

# Rename 'player_Id' to 'player_id'
hitters_data.rename(columns={'player_Id': 'player_id'}, inplace=True)

# Merge datasets on 'player_id' and 'Name'
merged_final_data = pd.merge(hitters_data, merged_success_data, on=['Name', 'player_id'], how='inner')
merged_final_data = pd.merge(merged_final_data, oppo_fb_data[['player_id', 'oppo_fb%']], on='player_id', how='left')
merged_final_data = pd.merge(merged_final_data, pulled_fb_data[['player_id', 'pulled_fb%']], on='player_id', how='left')
merged_final_data = pd.merge(merged_final_data, straightaway_fb_data[['player_id', 'straightaway_fb%']], on='player_id', how='left')
merged_final_data = pd.merge(merged_final_data, owar_data[['player_id', 'oWAR']], on='player_id', how='left')

# Define the columns to include in the correlation analysis
metrics = ['wRC+_x', 'HR_x', 'BB/K', 'ISO']

# Perform correlation analysis with oWAR
correlations = merged_final_data[metrics + ['oWAR']].corr()['oWAR'].drop('oWAR')
print("Correlation with oWAR:\n", correlations)

# Normalize the correlations to use as weights
weights = correlations / correlations.sum()
print("Weights for composite score:\n", weights)

# Calculate the weighted sum composite score
merged_final_data['Composite_Score'] = (
    merged_final_data['wRC+_x'] * weights['wRC+_x'] +
    merged_final_data['HR_x'] * weights['HR_x'] +
    merged_final_data['BB/K'] * weights['BB/K'] +
    merged_final_data['ISO'] * weights['ISO']
)


# Define the weights for each metric in the BONDS Index, including the composite score
bonds_weights = {
    'Brls/BBE%': 0.25,
    '50th_max_velo': 0.20,
    'O-Swing%_x': -0.15,
    'SwStr%_x': -0.10,
    'Z-Swing%': 0.05,
    'Z-Contact%_y': 0.10,
    'pulled_fb%': 0.15,
    'straightaway_fb%': 0.10,
    'oppo_fb%': -0.05,
    'Composite_Score': 0.50
}

# Calculate the BONDS Index for each player including the composite score
merged_final_data['BONDS_Index'] = (
    merged_final_data['Brls/BBE%'] * bonds_weights['Brls/BBE%'] +
    merged_final_data['50th_max_velo'] * bonds_weights['50th_max_velo'] +
    (1 - merged_final_data['O-Swing%_x']) * bonds_weights['O-Swing%_x'] +
    (1 - merged_final_data['SwStr%_x']) * bonds_weights['SwStr%_x'] +
    merged_final_data['Z-Swing%'] * bonds_weights['Z-Swing%'] +
    merged_final_data['Z-Contact%_y'] * bonds_weights['Z-Contact%_y'] +
    merged_final_data['pulled_fb%'] * bonds_weights['pulled_fb%'] +
    merged_final_data['straightaway_fb%'] * bonds_weights['straightaway_fb%'] +
    (1 - merged_final_data['oppo_fb%']) * bonds_weights['oppo_fb%'] +
    merged_final_data['Composite_Score'] * bonds_weights['Composite_Score']
)

# Apply the adjusted scaling for the BONDS Index
range_bonds_index = merged_final_data['BONDS_Index'].max() - merged_final_data['BONDS_Index'].min()
mean_bonds_index = merged_final_data['BONDS_Index'].mean()
merged_final_data['Adjusted Scaling'] = 50 + ((merged_final_data['BONDS_Index'] - mean_bonds_index) / range_bonds_index) * 50

bonds_index_leaderboard = merged_final_data[['Name', 'player_id', 'Adjusted Scaling']]
bonds_index_leaderboard.rename(columns={'Adjusted Scaling': 'BONDS Index'}, inplace=True)

# Save the final dataframe to a CSV file
output_file_path = "/Users/stephenak24/Downloads/BONDS_Index_Leaderboard.csv"
bonds_index_leaderboard.to_csv(output_file_path, index=False)


Correlation with oWAR:
 wRC+_x    0.861531
HR_x      0.620888
BB/K      0.467684
ISO       0.627497
Name: oWAR, dtype: float64
Weights for composite score:
 wRC+_x    0.334238
HR_x      0.240879
BB/K      0.181441
ISO       0.243442
Name: oWAR, dtype: float64


/var/folders/ft/2w6drxm14q597vm6scyknhc40000gn/T/ipykernel_34416/340781613.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bonds_index_leaderboard.rename(columns={'Adjusted Scaling': 'BONDS Index'}, inplace=True)


In [ ]:
# composite_weights = {
#    'wRC+_x': 0.334238,
#    'HR_x': 0.240879,
#    'BB/K': 0.181441,
#    'ISO': 0.243442